<a href="https://colab.research.google.com/github/dakyommii/study/blob/main/malnyunClone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

In [3]:
MODEL_SAVE_PATH = os.path.join("/content/drive/My Drive/results2")

In [4]:
DATA_PATH = "/content/drive/My Drive/atoi_gan_fin"

In [5]:
!git clone https://github.com/yu45020/Waifu2x.git

Cloning into 'Waifu2x'...
remote: Enumerating objects: 546, done.
remote: Total 546 (delta 0), reused 0 (delta 0), pack-reused 546
Receiving objects: 100% (546/546), 138.07 MiB | 32.42 MiB/s, done.
Resolving deltas: 100% (284/284), done.


In [6]:
import sys
sys.path.insert(0,"/content/Waifu2x")

In [7]:
!ls /content/Waifu2x

Common.py      examples		 Loss.py	     Readme.md
Dataloader.py  Img_to_Sqlite.py  model_check_points  train.py
docs	       LICENSE		 Models.py	     utils


In [ ]:
!mkdir out

In [ ]:
# for seed_idx, seed in enumerate(seeds):
#         print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
#         rnd = np.random.RandomState()
#         tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
#         images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
#         path = f"/content/drive/My Drive/results{seed_idx}.png"
#         PIL.Image.fromarray(images[0], 'RGB').save(path)

In [29]:
import utils
import PIL
from PIL import Image
# from keras.preprocessing.image import load_img

from utils.prepare_images import *
from Models import *
from torchvision.utils import save_image
model_cran_v2 = CARN_V2(color_channels=3, mid_channels=64, conv=nn.Conv2d,
                        single_conv_size=3, single_conv_group=1,
                        scale=2, activation=nn.LeakyReLU(0.1),
                        SEBlock=True, repeat_blocks=3, atrous=(1, 1, 1))
                        
model_cran_v2 = network_to_half(model_cran_v2)
checkpoint = "model_check_points/CRAN_V2/CARN_model_checkpoint.pt"
# model_cran_v2.load_state_dict(torch.load(checkpoint, 'fp16'))
# if use GPU, then comment out the next line so it can use fp16. 
model_cran_v2 = model_cran_v2.float() 

file_name_in_dir = []

for root,dirs,files in os.walk(DATA_PATH):
  for fname in files:
    full_fname=os.path.join(root,fname)
    file_name_in_dir.append(full_fname)

i = 0
for file_image in file_name_in_dir:
  i+=1
  # demo_img = load_img(file_image)
  # x=img_to_array(img)
  # x=x.reshape((1,)+x.shape)

  demo_img = f"/content/drive/My Drive/atoi_gan_fin/IMG{i}.png"
  img = Image.open(demo_img).convert("RGB")

  # origin
  img_t = to_tensor(img).unsqueeze(0) 

  # used to compare the origin
  img = img.resize((img.size[0] // 2, img.size[1] // 2), Image.BICUBIC) 

  # overlapping split
  # if input image is too large, then split it into overlapped patches 
  # details can be found at [here](https://github.com/nagadomi/waifu2x/issues/238)
  img_splitter = ImageSplitter(seg_size=64, scale_factor=2, boarder_pad_size=3)
  img_patches = img_splitter.split_img_tensor(img, scale_method=None, img_pad=0)
  with torch.no_grad():
      out = [model_cran_v2(k) for k in img_patches]
  img_upscale = img_splitter.merge_img_tensor(out)

  final = torch.cat([img_t, img_upscale])
  save_image(final, f'out{i}.png', nrow=2)


RuntimeError: ignored

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# import tensorflow as tf
# import sys
# import os

# sys.path.insert(0, "/content/atoiGan")

# !git clone https://github.com/bryandlee/FreezeG.git

fatal: destination path 'FreezeG' already exists and is not an empty directory.


In [ ]:
# ! python create_dataset.py --src_dir='train' --dst_path='"/content/drive/My Drive/results"' --is_include_only_pos

python3: can't open file 'create_dataset.py': [Errno 2] No such file or directory
